In [12]:
#Import dependencies
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup as bs
import requests
import pymongo

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News

In [2]:
#Visit NASA Mars news website and scrape headlines
nasa_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(nasa_url)
html = browser.html
soup = bs(html, 'html.parser')

In [3]:
# Retrieve the latest element that contains news title and news paragraph
article = soup.find("div", class_='list_text')
nasa_headline = article.find('div', class_='content_title').text
nasa_paragraph = article.find('div', class_='article_teaser_body').text

print(nasa_headline)
print(nasa_paragraph)


InSight Captures Sunrise and Sunset on Mars
InSight joins the rest of NASA's Red Planet surface missions, all of which have photographed either the start or end of a Martian day.


### JPL Mars Space Images 

In [4]:
#Visit JPL featured speace image website 
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)
html = browser.html
soup = bs(html, 'html.parser')

In [5]:
# Retrieve current Featured Mars Image 
image_url = soup.find("article", class_='carousel_item').a['data-fancybox-href']
featured_image_url = "https://www.jpl.nasa.gov" + image_url

print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19639_ip.jpg


### Mars Weather

In [6]:
#Visit Mars Weather twitter account
twitter_url ="https://twitter.com/marswxreport?lang=en"
browser.visit(twitter_url)
html = browser.html
soup = bs(html, 'html.parser')

In [33]:
#Retrieve current Mars weather report
tweet = soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")
mars_weather = tweet.text.strip()
print(mars_weather)

InSight sol 153 (2019-05-02) low -98.5ºC (-145.3ºF) high -17.5ºC (0.6ºF)
winds from the SW at 4.7 m/s (10.5 mph) gusting to 11.9 m/s (26.6 mph)
pressure at 7.40 hPapic.twitter.com/5RdVDek7mh


### Mars Facts

In [23]:
#Visit Mars Facts webpage and use Pandas to scrape fact table
import pandas as pd 

url = "https://space-facts.com/mars/"
marsfact_df = pd.read_html(url)[0]
marsfact_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [24]:
#Convert to dataframe and clean up
marsfact_df = marsfact_df.rename(columns={0:"Parameter",1:"Value"})
marsfact_df.set_index("Parameter", inplace=True)
marsfact_df

,Value
Parameter,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [25]:
#Convert to html table string
marsfact_html = marsfact_df.to_html()
marsfact_html= marsfact_html.replace('\n','')
marsfact_html

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Parameter</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

### Mars Hemispheres

In [11]:
#Visit USGS Astrogeology site to scrape image
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemi_url)
html = browser.html
soup = bs(html, 'html.parser')

In [12]:
#List of hemispheric names with links for the images
hemi_list =[]

hemi_all = soup.find_all("h3")

for hemi in hemi_all:
    hemi_list.append(hemi.text)

hemi_list

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [13]:
hemisphere_image_urls = []
hemi_dic = {}

for hemi in hemi_list:
    
    browser.click_link_by_partial_text(hemi)
    html = browser.html
    soup = bs(html, 'html.parser')
    img = soup.find("div", class_='downloads')
    link = img.find('a')
    url = link['href']
    
    hemi_dic["title"] = hemi
    hemi_dic["img_url"] = url
    hemisphere_image_urls.append(hemi_dic)
    
    browser.back()
    
print(hemisphere_image_urls)

[{'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
